# L3 Convolutional Neural Networks

## Improvements based on form of data

Can do better if you know what the form your data is, e.g. words or an image. 
Idea: If your data has structure and your network does not have to learn that structure from scratch, it will do better.

E.g. 1: Model that ignores colour when classifying pictures of letters will do better because colour doesn't matter. So ask model to only consider greyscale $$\frac{R+G+B}{3}$$ rather than R,G,B.

E.g. 2: You have an image and you want your network to say it's the image of a cat. It doesn't matter where the kitten is. You can use **translational invariance** to make things easier. It'll be tough if your network has to relearn what a cat looks like if it's in the bottom left corner vs if it's in the top right corner.

E.g. 3: You have a paragraph of text. The meaning of 'kitten' probably won't be different depending on which sentence it's in. So you might want the part of the network that learns what a kitten is to be reused every time you see the word 'kitten' as opposed to having it relearn every time.

We can do this in neural networks via **weight sharing**.
* When you know that two inputs can contain the same kind of info, you share their weights and train the weights jointly for those inputs.

> Weight sharing not only reduces variance (fewer parameters) but also helps learning with spatial invariance.

It's important: **Statistical invariants** (things that don't change on average across time and space) are everywhere.

Consequences:
* Images -> Convolutional neural networks
* Text -> Embeddings and recurrent neural networks

## Convolutional Neural Networks
Neural networks that share parameters across space. They are also called **covnets**.

E.g. we have an image that can be represented as a flat pancake. It has a width, height (same as width and height of image) and a depth. 
* Depth: because the image is usually coloured, it can have a depth of 3, covering red, blue and green channels).

Imagine taking a small patch of this image and running a tiny neural network on it with k outputs. If you represent the outputs vertically in a tiny column. 

Then slide the neural network across the image without changing the weights across and vertically, as though you're painting the image with a brush.

On the output we've drawn another image with a different  width, height and depth (Depth is k). This operation is called a convolution. 

If the patch size were the size of the entire image, we would have a normal neural network. But because the patch size is small, we have many fewer weights and they are shared across space.

Deep networks where we have stacks of convolutions instead of stacks of matrix multiplication layers.

### Convolutional pyramid
Base: have layer that is shallow.

Apply convolutions that squeeze spatial dimensions while increasing the depth, which corresponds roughly to the semantic complexity of your representation. Can put classifier on top when you have squeezed out all spatial parameters and only parameters that map to content of the image remain.

Q: If the cat takes up the entire image, does this not cause problems? 

A: This should not cause problem since the size of receptive field will grow given a deep enough NN.

* **Kernels**: patches.
* **Feature map**: Each pancake (depth) in your stack.
* **Stride**: The number of pixels you're shifting when you move your filter.  Stride of 1 means same size. Stride of 2 roughly half the size. 'Roughly' because it depends what you do at the edge of your image .
    * **'Valid'** padding: Don't go off the edge
    * **'Same'** padding: Go off the edge and pad with zeroes so the output map size is the exactly the same size as the input map.

Q: What are the width, depth and height of the output for a 28x28 pixel image with a patch size of 3x3, input depth of 3 and output depth of 8 if we use
* Same padding with stride 1
* Valid padding with stride 1
* Valid padding with stride 2

(A: 28x28, 26x26, 13x13 all with depth 8)

### Building a covnet
* Stack up convolutions, then use strides to reduce dimensionality and increase depth of your network layer after layer.
* Connect whole thing to fully connected layers.
* Ready to train classifier

**What happens to the Chain Rule with weight sharing**: Nothing really. Just add the gradients for every patch (all possible locations on the image).

## Covnet Improvements

### Pooling
Better way to reduce spatial dimensions.
Striding by, say, 2+ to reduce feature map size is aggressive and removes much information. 

**Pooling** combines all convolutions in a neighbourhood (instead of skipping e.g. one in two convolutions).

1. **Max Pooling**
At every point on the feature map, look at a small neighbourhood around that point and compute the max of all the responses around it.
    * Parameter-free -> doesn't add to number of parameters -> don't risk additional overfitting
    * Often more accurate
    * More expensive to compute (Because convolutions run at lower stride)
    * More hyperparameters: pooling size, pooling stride. They don't have to be the same.

**Typical architecture of covnets: Alternating convolutions and max pooling followed by a few fully connected layers at the top.** E.g. 'Le Net-5' by Yann Lecun in 1998, 'Alexnet' Alex Kaizhevsky 2012.


2. **Average Pooling**
Take average of pixels of window of pixels around that point instead of the max.
Like providing a blurred, low resolution view of the feature map below it.

### 1x1 Convolutions

Looking at one pixel only. Convolution is basically a linear classifier for a patch of the image.

Interspersing convolutions with 1x1 convolutions is a cheap way to make your models deeper without completely changing their structure.

They're cheap because they're actually matrix multiplications and have few parameters.

### Inception Module
Here is a quick introduction of the technqiue on [Youtube](https://www.youtube.com/watch?v=VxhSouuSZDY)

General strategy that has been successful at creating smaller and better covnets than pure convolutional pyramids

Idea: At each layer of your covnet, you can make a choice: have a pooling operation or a convolution (then what size convolution?). All these are beneficial to the modeling power of your network. So let's use them all. Have
* 1x1 then 5x5
* 1x1 then 3x3
* 1x1
* Average pooling then 1x1
Concatenate output.

You can choose these parameters in such a way that the total number of parameters in your model is v small, and yet the model performs better than it would if you only had a simple convolution.